## Imports

In [7]:
import pandas as pd
import json
import time
import random
import requests
from dotenv import load_dotenv
import os
from tqdm import tqdm
load_dotenv()

True

## Configs

In [8]:
#==Config==#
OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")

## Load Data

In [9]:
df = pd.read_csv("D:\Projects\Personal\Hackathon\TwinSpherev2\PreprocessingScripts\grouped_concatenated_tweets.csv")

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\tejas\AppData\Local\Temp\ipykernel_25012\3661820383.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_csv("D:\Projects\Personal\Hackathon\TwinSpherev2\PreprocessingScripts\grouped_concatenated_tweets.csv")


In [10]:
df.head()

,name,concatenated_tweets
0,10Ronaldinho,b'Baita recep\xc3\xa7\xc3\xa3o dessa galera an...
1,10neto,b'@andreolifelipe A\xc3\xad brigam com a gente...
2,143redangel,b'Julie Yap-Daza = huli (To get caught). \n\ne...
3,1LoganHenderson,"b""Check this out! @LOWDNoizez' debut single of..."
4,1victorvaldes,b'\xd8\xb4\xd8\xb1\xd9\x83\xd8\xa9 \xd8\xa7\xd...


## Helper Functions

In [11]:
# Prompt Template
def build_prompt(tweet_block):
    return f"""
Analyze the following Twitter content from a user. Based on their language, common themes, and overall behavior expressed in these tweets, infer the following personality traits. Output the results as a detailed persona paragraph.

1.  **Interests/Topics:** List 3-5 keywords or phrases describing their main interests and the topics they discuss.
2.  **Communication Style & Tone:** Describe how they express themselves.
3.  **General Sentiment:** Describe their overall emotional leaning.
4.  **Engagement Patterns:** Describe how they interact on the platform.

Ensure your output is a paragraph detailing the persona, using the points mentioned above. There should be nothing apart from this paragraph in your output.

The following Content section contains the latest 100 tweets of the user concatenated together as a string and seperated by the seperator "<ENDOFTWEET>"

Content: {tweet_block}
""".strip()

In [12]:
# Function to call OpenRouter with DeepSeek
def query_openrouter(prompt, model="deepseek/deepseek-chat-v3-0324:free"):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPEN_ROUTER_API_KEY}",
            "Content-Type": "application/json"
        },
        data=json.dumps({
            "model": "deepseek/deepseek-chat-v3-0324:free",
            "messages": [{
                "role": "user",
                "content": prompt
            }],
            
        })
    )

    if response.status_code != 200:
        raise Exception(f"API error {response.status_code}: {response.text}")

    result = response.json()
    return result["choices"][0]["message"]["content"]

## Extract persona

In [ ]:
output_data = []

sampled_df = df.sample(10)

# Iterate through rows and generate personas
for _, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc="Extracting Personas"):
    name = row["name"]
    tweets = row["concatenated_tweets"]

    try:
        prompt = build_prompt(tweets)
        response = query_openrouter(prompt)
        output_data.append({"name": name, "persona": response})
        time.sleep(1.5)  # Avoid rate-limiting
    except Exception as e:
        print(f"[!] Error processing {name}: {e}")
        continue

Extracting Personas: 100%|██████████| 10/10 [02:11<00:00, 13.16s/it]

[{'name': 'LucianoHuck', 'persona': 'This Twitter user presents as an engaged, socially conscious individual with a dynamic and expressive communication style. Their **interests/topics** revolve around **social justice and inequality** (highlighting favelas, Covid-19 disparities), **culture and arts** (support for creative economies, tributes to artists), and **Brazilian politics** (critical discussions on governance, polarization). They frequently share personal experiences, event promotions, and interactions with public figures, suggesting a mix of activism and media engagement. Their **communication style** is warm, enthusiastic, and occasionally playful (using emojis, humor, and hashtags like *#latavelha*), but shifts to earnest and persuasive when addressing serious topics. The **general sentiment** is **optimistic and empathetic**, balancing gratitude, excitement for collaborations, and distress over societal issues. They exhibit high **engagement patterns**, actively retweeting,

In [14]:
pd.DataFrame(output_data).to_csv("D:\Projects\Personal\Hackathon\TwinSpherev2\Data\Persona.csv")

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\tejas\AppData\Local\Temp\ipykernel_25012\3265518971.py:1: SyntaxWarning: invalid escape sequence '\P'
  pd.DataFrame(output_data).to_csv("D:\Projects\Personal\Hackathon\TwinSpherev2\Data\Persona.csv")
